# Data prepartion and model training



## NOTE : THIS PART OF THE CODE SNIPPETS ARE THE PART OF THE INFORMATION TECHNOLOGY PROJECT OF  HTET AUNG (522K0048) AND HSU MYAT WAI MAUNG (522K0049)  FROM FACULTY OF INFORMATION TECHNOLOGY UNDER THE SUPERVISION OF TON DUC THANG UNIVERSITY.



- In order to train the YOLOv11 model we need to make sure we have the yolov11s.yaml in the  /content/. directory in the google colab.


- Remember to download the latest trained version from /run/detection./traffic signs detection/ weights/ last.pt

- Check that we have the neccessary files to resume the training once the training stopped.






In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.7 MB/s eta 0:00:00


In [ ]:
import os
import zipfile
import logging
import torch
import yaml
import shutil
from datetime import datetime
from ultralytics import YOLO
from google.colab import drive

# ==============================================================================
# 1. SETUP LOGGING
# ==============================================================================
log_dir = '/content/logs'
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, f'training_log_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger('YOLOv11_Training_Pipeline')

logger.info("Starting YOLOv11 training pipeline on Google Colab.")

# ==============================================================================
# 2. MOUNT GOOGLE DRIVE & UNZIP DATASET
# ==============================================================================
def setup_dataset_from_drive(zip_name='merged_dataset.zip'):
    """
    Mounts Google Drive, locates the dataset zip file, and extracts it.
    """
    try:
        drive.mount('/content/drive', force_remount=True)
        drive_root = '/content/drive/MyDrive'
        logger.info(f"Google Drive mounted successfully at {drive_root}.")
    except Exception as e:
        logger.error(f"Failed to mount Google Drive: {str(e)}")
        raise RuntimeError("Google Drive mount failed. Please authorize access.")

    zip_file_path = os.path.join(drive_root, zip_name)
    extract_dir = '/content/'

    if not os.path.exists(zip_file_path):
        raise FileNotFoundError(f"Dataset zip file '{zip_name}' not found in Google Drive.")

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    logger.info("Dataset extracted successfully to /content/.")
    return drive_root

try:
    gdrive_root = setup_dataset_from_drive()
except (RuntimeError, FileNotFoundError) as e:
    logger.critical(f"Setup failed: {e}")


# ==============================================================================
# 3. VERIFY AND FIX DATASET CONFIGURATION
# ==============================================================================
dataset_root_path = '/content/merged_dataset'
data_yaml_path = os.path.join(dataset_root_path, 'data.yaml')

def fix_yaml_paths(yaml_path, dataset_root):
    """
    Corrects the paths in the data.yaml file to be absolute for the Colab environment.
    """
    if not os.path.exists(yaml_path):
        raise FileNotFoundError(f"data.yaml not found at {yaml_path}. Check your zip file structure.")

    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    data['train'] = os.path.join(dataset_root, 'images', 'train')
    data['val'] = os.path.join(dataset_root, 'images', 'val')
    data['test'] = os.path.join(dataset_root, 'images', 'test')

    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)
    logger.info(f"Successfully corrected paths in {yaml_path}")

fix_yaml_paths(data_yaml_path, dataset_root_path)


# ==============================================================================
# 4. TRAIN THE YOLOv11 MODEL
# ==============================================================================
def train_model(data_yaml, resume_training=False):
    """
    Trains the YOLOv11 model, with an option to resume from the last checkpoint.
    """
    run_name = 'traffic_sign_model_continued_run'

    if resume_training:
        model_path = '/content/last (4).pt'

        if not os.path.exists(model_path):
            error_message = (
                f"Resume checkpoint not found at '{model_path}'.\n"
                "Please make sure the file 'last_yolo11.pt' exists in your Google Drive."
            )
            raise FileNotFoundError(error_message)

        logger.info(f"Resuming training from checkpoint: {model_path}.")
        model = YOLO(model_path)
    else:
        model_config_path = 'yolov11s.yaml'
        if not os.path.exists(model_config_path):
            raise FileNotFoundError(f"Model config '{model_config_path}' not found. Please upload it to your Colab session.")

        logger.info(f"Starting a new training run from scratch using '{model_config_path}'.")
        model = YOLO(model_config_path)

    device_to_use = 0 if torch.cuda.is_available() else 'cpu'
    logger.info(f"Training device selected: {'GPU' if device_to_use == 0 else 'CPU'}")

    results = model.train(
        data=data_yaml,
        epochs=50,
        imgsz=640,
        batch=16,
        name=run_name,
        patience=10,
        optimizer='AdamW',
        lr0=0.001,
        amp=True,
        device=device_to_use,
        augment=True,
        resume=resume_training
    )
    logger.info("Training completed successfully.")
    return model

# ==============================================================================
# 5. SAVE RESULTS TO GOOGLE DRIVE
# ==============================================================================
def save_results_to_drive(model, drive_root_path):
    """
    Copies the entire results folder to Google Drive for permanent storage.
    """
    try:
        source_dir = model.trainer.save_dir
        run_name = os.path.basename(source_dir)
        destination_dir = os.path.join(drive_root_path, 'YOLOv11_Training_Results', run_name)

        if os.path.exists(destination_dir):
            shutil.rmtree(destination_dir)

        logger.info(f"Copying results from {source_dir} to {destination_dir}...")
        shutil.copytree(source_dir, destination_dir)
        shutil.copy(log_file, destination_dir)
        logger.info("Successfully saved all results and logs to Google Drive.")

    except Exception as e:
        logger.error(f"Failed to save results to Google Drive: {str(e)}", exc_info=True)


trained_model = train_model(data_yaml_path, resume_training=True)

save_results_to_drive(trained_model, gdrive_root)

logger.info("--- YOLOv11 Training Pipeline Completed ---")



Mounted at /content/drive
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/merged_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/last (4).pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=traffic_sign_model_final_from_scratch, nbs=64, nms=False, opset=None, optimize=False, optimizer=Ada

In [ ]:
!yolo task=detect mode=val \
    model=/content/runs/detect/traffic_sign_model_final_from_scratch/weights/best.pt \
    data=/content/merged_dataset/data.yaml \
    split=test \
    name=YOLOv11_FINAL_TEST_EVALUATION

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv11s summary (fused): 72 layers, 11,158,866 parameters, 0 gradients, 28.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 13.6±15.1 MB/s, size: 24.8 KB)
val: Scanning /content/merged_dataset/labels/test... 2119 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2119/2119 1.3Kit/s 1.6s
val: New cache created: /content/merged_dataset/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 133/133 4.6it/s 29.1s
                   all       2119       2757      0.905      0.928      0.936      0.829
           speed_limit        517        517       0.99      0.986      0.995      0.955
       end_speed_limit         34         34      0.983          1      0.995      0.958
            no_passing        124        124      0.974      0.898      0.943       0.82
     no_heavy_vehicles        104        104      0.981      0.98

In [ ]:
!yolo task=detect mode=predict \
    model=/content/runs/detect/traffic_sign_model_final_from_scratch/weights/best.pt \
    conf=0.4 \
    source=/content/merged_dataset/images/test/ \
    name=YOLOv11_PREDICTIONS \
    save=True

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv11s summary (fused): 72 layers, 11,158,866 parameters, 0 gradients, 28.6 GFLOPs

image 1/2119 /content/merged_dataset/images/test/gtsrb_00006.png: 576x640 1 general_caution, 45.3ms
image 2/2119 /content/merged_dataset/images/test/gtsrb_00009.png: 640x608 1 ahead_only, 47.4ms
image 3/2119 /content/merged_dataset/images/test/gtsrb_00018.png: 640x640 1 pedestrians, 16.9ms
image 4/2119 /content/merged_dataset/images/test/gtsrb_00027.png: 640x640 1 no_heavy_vehicles, 16.4ms
image 5/2119 /content/merged_dataset/images/test/gtsrb_00035.png: 640x640 1 no_entry, 16.4ms
image 6/2119 /content/merged_dataset/images/test/gtsrb_00042.png: 576x640 1 general_caution, 17.0ms
image 7/2119 /content/merged_dataset/images/test/gtsrb_00043.png: 640x640 1 yield, 17.2ms
image 8/2119 /content/merged_dataset/images/test/gtsrb_00053.png: 640x640 1 no_passing, 14.5ms
image 9/2119 /content/merged_dataset/images/test/gtsrb_00058

In [ ]:
!pip install gradio

import gradio as gr
from ultralytics import YOLO
from PIL import Image
import os

# --- 1. LOAD YOUR MODEL ---
# Make sure this path points to your BEST model
model_path = '/content/runs/detect/traffic_sign_model_final_from_scratch/weights/best.pt'
model = YOLO(model_path)

# --- 2. DEFINE PREDICTION FUNCTION ---
def predict_image(input_image):
    """Runs prediction on the input image using the YOLOv11 model."""
    results = model.predict(input_image)
    result_image_bgr = results[0].plot()
    result_image_rgb = Image.fromarray(result_image_bgr[..., ::-1])
    return result_image_rgb

# --- 3. CREATE THE GRADIO INTERFACE ---
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload Your Test Image"),
    outputs=gr.Image(type="pil", label="YOLOv11 Result"),
    title="YOLOv11 Traffic Sign Detection Demo",
    description="Test the model you trained in your project. Upload any image to see the detected traffic signs.",
    examples=[
        ['/content/merged_dataset/images/test/image1.jpg'],
        ['/content/merged_dataset/images/test/image2.jpg']
    ]
)

# --- 4. LAUNCH THE APP ---
iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5ef557f0e49d889ea1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 640x480 (no detections), 45.1ms
Speed: 2.1ms preprocess, 45.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)
Created dataset file at: .gradio/flagged/dataset1.csv

0: 448x640 1 one_way, 1 traffic_queues, 46.1ms
Speed: 2.5ms preprocess, 46.1ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5ef557f0e49d889ea1.gradio.live


In [ ]:
!zip -r /content/my_project_results.zip /content/runs/

  adding: content/runs/ (stored 0%)
  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/traffic_sign_model_final_from_scratch/ (stored 0%)
  adding: content/runs/detect/traffic_sign_model_final_from_scratch/BoxF1_curve.png (deflated 9%)
  adding: content/runs/detect/traffic_sign_model_final_from_scratch/val_batch0_pred.jpg (deflated 17%)
  adding: content/runs/detect/traffic_sign_model_final_from_scratch/confusion_matrix_normalized.png (deflated 19%)
  adding: content/runs/detect/traffic_sign_model_final_from_scratch/args.yaml (deflated 53%)
  adding: content/runs/detect/traffic_sign_model_final_from_scratch/confusion_matrix.png (deflated 20%)
  adding: content/runs/detect/traffic_sign_model_final_from_scratch/results.png (deflated 7%)
  adding: content/runs/detect/traffic_sign_model_final_from_scratch/val_batch2_labels.jpg (deflated 16%)
  adding: content/runs/detect/traffic_sign_model_final_from_scratch/val_batch1_pred.jpg (deflated 13%)
  adding: content/runs/de